In [4]:
!pip install mne
!pip install hdf5storage

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 2.5 MB/s eta 0:00:00


In [5]:
import numpy as np
import mne
import scipy.io as sio
import hdf5storage
import os
import math


In [ ]:
#Test with a single night

path_raw_data = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple/S_01/night_1/EEG_raw.mat'
path_cleaned_data = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple_cleaned/S_01/night_1/EEG_raw.mat'

print('loading data')
data_raw = hdf5storage.loadmat(path_raw_data)
print("raw data loaded")

#data_cleaned = hdf5storage.loadmat(path_cleaned_data)['data']
#print("clean data loaded")

In [ ]:
print('debug')

In [ ]:

labels = data_raw['chanlabels'].reshape(25).tolist() #channel labels needed to load data with MNE

# A line to first load a nights data into mne, then resample to 250 hz, then get the numpy array out again with ._data
# Data is converted to 32 bit
data = mne.io.RawArray(data_raw['data'], mne.create_info(labels,500)).resample(250)._data.astype("float32")

In [ ]:
# This section processes the data, and creates files inside the original folder structure.

clean_data_dir = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple_cleaned'
raw_data_dir = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple'

# Run through all the cleaned EEG files
for subdir, dirs, files in os.walk(raw_data_dir):
    for file in files:
        if "EEG" in file and not "250hz" in file:
            print(os.path.join(subdir, file))
            data_raw = hdf5storage.loadmat(os.path.join(subdir, file))
            print('Raw data loaded')

            # Labels are needed for MNE. Imported, reshaped and converted to list
            labels = data_raw['chanlabels'].reshape(25).tolist() #channel labels needed to load data with MNE


            # First load a nights data into mne, then resample to 250 hz, then get the numpy array out again with ._data
            # Data is converted to 32 bit
            data = mne.io.RawArray(data_raw['data'], mne.create_info(labels,500)).resample(250)._data.astype("float32")
            print('downsampling complete')

            # change NaN to 0, and save downsampled data, and add to new file
            np.save(f'{os.path.join(subdir)}/EEG_raw_250hz', np.nan_to_num(data))

            print(f'Done: {subdir}')